### 1.评论处理

In [11]:
import re
import os
import pickle
import jieba.analyse
import jieba.posseg as pseg
import math
import pandas as pd  
import opencc
jieba.setLogLevel(jieba.logging.INFO)

In [12]:
FH_PUNCTUATION = [
    (u"。", u"."), (u"，", u","), (u"！", u"!"), (u"？", u"?"), (u"～", u"~"),
]

keep_p = ['，', '。', '！', '？', '～', '、']
f2h = {}
for item in FH_PUNCTUATION:
    c1 = item[0]
    c2 = item[1]
    f2h[c2] = c1

导入评论,去除重复评论

In [13]:
# 读取 CSV 文件
df = pd.read_csv('../data/XiFaShui(11061).csv',header=None)  
print(len(df))
# 去除重复的评论  
unique_comments = df[0].drop_duplicates()  
print(len(unique_comments))  
with open('../data(temp)/unique_comments.txt', 'w', encoding='utf-8') as f:  
    for comment in unique_comments:  
        comment=comment.strip()
        if(len(comment)<=3):
            continue
        f.write(comment+'\n') 

# 输出去重后的评论  
print(unique_comments[:10])  

16403
5259
0     朋友给我推荐的洗发水，针对出油严重扁塌很有效。洗完后，头发立马蓬松了，两三天都不会油，整个发...
1      9天后追评：和我之前的洗发水相比，这款洗发水真的太好用了。控油效果特别的好。他们家洗护产品都推荐
2                                 发货很快，已经用了2次，还不错！值得回购！
6     本人头发很油，用了不错，这款洗发水味道也挺好闻的，买了一套，还送了好多小样，挺好的，用完再买...
7                             你敢想性价比这么高的洗发水吗？味道很好闻！很满意！
8                                             第二次回购了，不错
9               真的特别好用，爱了爱了，洗完超级清爽，特别舒服，高端大气，值得购买，物流也快。
10    真的超爱我的蓬松又巨显发量的头发，这都多亏了这款洗发水的功劳，它的起泡力很牛，打出来的泡沫超...
11    9天后追评：也是出门被追着问的一天，今天跟朋友小聚了一下，都在问我用的什么洗发水，不仅味道香...
12    洗前洗后，真的区别好大，我经常烫染头发，还喜欢调色。之前头发不管怎么洗，都老是贴着头皮。这款...
Name: 0, dtype: object


导入停用词

In [14]:
stop_word = []
with open('../data/stopword.txt', 'r',encoding='utf-8') as f:
    for line in f.readlines():
        stop_word.append(line.strip())
jieba.load_userdict('../data/user_dict.txt')#导入自定义词典,增强识别能力_1

繁体转为简体

In [15]:
def convert_file_to_simplified(file_path):  
    # 创建繁转简的转换器  
    converter = opencc.OpenCC('hk2s')  # 繁体到简体  

    # 读取文件内容  
    with open(file_path, 'r', encoding='utf-8') as file:  
        content = file.read()  

    # 转换内容  
    simplified_content = converter.convert(content)  

    # 保存回原文件  
    with open(file_path, 'w', encoding='utf-8') as file:  
        file.write(simplified_content)  

path='../data(temp)/unique_comments.txt'
convert_file_to_simplified(path)

清洗文本

In [16]:
def convert(content):
#英文标点转中文标点
    nc = []
    for c in content:
        if c in f2h:
            nc.append(f2h[c])
            continue
        nc.append(c)
    return "".join(nc)

def clean(line):
    if line == "":
        return line
    line = convert(line)
    c_content = []
    for char in line:
        if re.search("[\u4e00-\u9fa5]", char):
            c_content.append(char)
        elif re.search("[a-zA-Z0-9]", char):
            c_content.append(char)
        elif char in keep_p:
            c_content.append(char)
        elif char == ' ':  # 很多用户喜欢用空格替代标点
            c_content.append('，')
        else:
            c_content.append('')
    nc_content = []
    c = 0
    for char in c_content:
        if char in keep_p:
            c += 1
        else:
            c = 0
        if c < 2:
            nc_content.append(char)
    result = ''.join(nc_content)
    result = result.strip()
    result = result.lower()  # 所有英文转成小写字母
    return result

def clean_review(text):
#对原始评论进行清理，删去非法字符，统一标点，删去无用评论
    review_set = []
    for line in text:
        line = line.lstrip()
        line = line.rstrip()
        line = clean(line)
        if len(line) < 7:  # 过于短的评论需要删除
            continue
        if line and line not in ['该用户没有填写评论。', '用户晒单。']:
            review_set.append(line)
    return review_set

In [17]:
with open(path, 'r',encoding='utf-8') as f_r:
    text = f_r.readlines()
review_set = clean_review(text)

with open('../data(temp)/review_cleaned.txt', 'w',encoding='utf-8') as f_w:
    for i in review_set:
        f_w.write(i+'\n')

对清洗后的文本进行分词，词性标注

In [18]:
def get_seg_pos(text, type='word'):
#获取文档的分词以及词性标注结果，分词的方式可以为按词切分或者按字切分
    if type == 'word':
        seg_pos = []
        for line in text:
            line = line.strip()
            line_cut = pseg.cut(line)
            wordlist = []
            for term in line_cut:
                wordlist.append('%s/%s' % (term.word, term.flag))
            seg_pos.append(' '.join(list(wordlist)))

        return seg_pos
    
with open('../data(temp)/review_cleaned.txt', 'r',encoding='utf-8') as f_r:
    text = f_r.readlines()
seg_pos = get_seg_pos(text)
with open('../data(temp)/review_seg_post.txt', 'w',encoding='utf-8') as f_w:
    for i in seg_pos:
        f_w.write(i+'\n')

算除去停用词的每个词的idf值

In [19]:
def caculate_word_idf(seg_post_path, stop_word):
#计算所有文档中的每个词的idf
#seg_post_path: str, 分词路径
#stop_word: list, 停用词list
#return: 所有词的idf值
    num_doc = 0  # 商品数量
    word_IDF = {}  # word-IDF 记录每个word在不同的doc出现过的次数,然后计算IDF
    cur_doc_word_set = set()  # 记录当前文档中出现的不同的词
    for _, line in enumerate(open(seg_post_path, 'r',encoding='utf-8')):
        num_doc=num_doc+1
        line = line.strip()
        word_list = line.split(" ")
        word_list = [term.split('/')[0] for term in word_list]
        for w in word_list:
            # 如果这个词在停用词表中就不添加
            if w in stop_word:
                continue
            cur_doc_word_set.add(w)
    for w in cur_doc_word_set:
        if w in word_IDF:
            word_IDF[w] += 1
        else:
            word_IDF[w] = 1
    print(word_IDF)
    for w in word_IDF:
        word_IDF[w] = math.log10(num_doc / word_IDF[w])
    return word_IDF

word_idf = caculate_word_idf('../data(temp)/review_seg_post.txt', stop_word)
with open('../data(temp)/review_idf.txt', 'w', encoding='utf-8') as f:
    for word in word_idf:
        f.write('%s %s\n' % (word, word_idf[word]))

{'想要': 1, '仿佛': 1, '各具特色': 1, '每一款': 1, '超值': 1, '超顺': 1, '涂': 1, '截图': 1, '赔钱': 1, '沐浴露': 1, '每瓶': 1, '连夜': 1, '到用': 1, '最最': 1, '差': 1, '变痒': 1, '四五年': 1, '淡淡': 1, '加': 1, '同意': 1, '加购': 1, '负责': 1, '福星': 1, '气味': 1, '很强': 1, '完发': 1, '真不错': 1, '保留': 1, '好像': 1, '形容': 1, '贵': 1, '实足': 1, '特地': 1, '心头': 1, '款': 1, '五六年': 1, '敏感': 1, '重点': 1, '不见': 1, '清雅': 1, '美': 1, '追评': 1, '浓密': 1, '挑剔': 1, '女孩': 1, '赶紧': 1, '装用': 1, '评价': 1, '短发': 1, '清除': 1, '凑热闹': 1, '两': 1, '少许': 1, '次数': 1, '言': 1, '发货': 1, '本命': 1, '那款': 1, '洗掉': 1, '婷婷': 1, '女神': 1, '毫不逊色': 1, '首次': 1, 'kk': 1, '购买': 1, '卡厘': 1, '洗发': 1, '抽干': 1, '19': 1, '毫不犹豫': 1, '发泡': 1, '全程': 1, '吸管': 1, '正': 1, '169': 1, '细致': 1, '五天': 1, '头': 1, '利用': 1, '容量': 1, '痛': 1, '感': 1, '一瓶': 1, '不用': 1, '冠军': 1, '同款': 1, '够买': 1, '扁塌': 1, '清水': 1, '妆': 1, '每种': 1, '挺不错': 1, '64': 1, '买不到': 1, '确认': 1, '头油': 1, '寄过去': 1, '适合': 1, '官旗': 1, '换成': 1, '氯化钠': 1, '舒适': 1, '衣服': 1, '点': 1, '清爽型': 1, '真没想到': 1, '从未有过': 1, '淡': 1, '自信': 1, '会长': 1, '半

In [20]:
import csv  
# 将数据写入CSV文件  
with open('../data(temp)/idf.csv', 'w', newline='', encoding='utf-8') as csvfile:  
    writer = csv.writer(csvfile)  
    writer.writerow(['word', 'idf'])  # 写入表头  
    for word,idf in word_idf.items():   
        writer.writerow([word, f'{idf:.4f}'])  # 写入合成词和格式化的得分  

### 2.基于PIM+左右熵发现新词
需根据综合得分,手动把合适的词加入用户词典(userdict)

In [21]:
import math

WINDOW_SIZE = 5
PUNCTUATION_MARK = ['x']  # 标点
NOUN_MARK = ['n', 'ng', 'nr', 'nrfg', 'nrt', 'ns', 'nt', 'nz']  # 名词
VERB_MARK = ['v', 'vd', 'vg', 'vi', 'vn', 'vq']  # 动词
ADJECTIVE_MARK = ['a', 'ad', 'an', 'ag']  # 形容词
ADVERB_MARK = ['d', 'df', 'dg']  # 副词
ENG_MARK = ['eng']

RESERVED_MARK = NOUN_MARK + VERB_MARK + ADJECTIVE_MARK + ADVERB_MARK + ENG_MARK # 用于发现新词


def text2review(seg_pos_text):
#经过分词的文档，得到原始用户的每条评论
    review_list = []  # 保存全部的按照指定标点切分的句子
    all_word = set()  # 全部单词
    for line in seg_pos_text:
        line = line.strip()
        line = line.split(' ')
        cur_review = []
        for term in line:
            word, flag = term.split('/')
            cur_review.append(word)
            if flag in RESERVED_MARK:
                all_word.add(word)
        review_list.append(cur_review)

    return review_list, all_word


def find_word_phrase(all_word, seg_list):
#根据点互信息以及信息熵发现词组，主要目的是提升分词效果
    word_count = {k: 0 for k in all_word}  # 记录全部词出现的次数

    all_word_count = 0
    all_bi_gram_count = 0
    for sentence in seg_list:
        all_word_count += len(sentence)
        all_bi_gram_count += len(sentence) - 1
        for idx, word in enumerate(sentence):#idx似乎没有用到
            if word in word_count:
                word_count[word] += 1

    bi_gram_count = {}
    bi_gram_lcount = {}
    bi_gram_rcount = {}
    for sentence in seg_list:
        for idx, _ in enumerate(sentence):
            left_word = sentence[idx - 1] if idx != 0 else ''
            right_word = sentence[idx + 2] if idx < len(sentence) - 2 else ''

            first = sentence[idx]
            second = sentence[idx + 1] if idx + 1 < len(sentence) else ''
            if first in word_count and second in word_count:
                if (first, second) in bi_gram_count:
                    bi_gram_count[(first, second)] += 1
                else:
                    bi_gram_count[(first, second)] = 1
                    bi_gram_lcount[(first, second)] = {}
                    bi_gram_rcount[(first, second)] = {}

                if left_word in bi_gram_lcount[(first, second)]:
                    bi_gram_lcount[(first, second)][left_word] += 1
                elif left_word != '':
                    bi_gram_lcount[(first, second)][left_word] = 1

                if right_word in bi_gram_rcount[(first, second)]:
                    bi_gram_rcount[(first, second)][right_word] += 1
                elif right_word != '':
                    bi_gram_rcount[(first, second)][right_word] = 1

    bi_gram_count = dict(filter(lambda x: x[1] >= 5, bi_gram_count.items()))#过滤词频<5的词

    bi_gram_le = {}  # 全部bi_gram的左熵
    bi_gram_re = {}  # 全部bi_gram的右熵
    #计算左熵
    for phrase in bi_gram_count:
        le = 0
        for l_word in bi_gram_lcount[phrase]:
            p_aw_w = bi_gram_lcount[phrase][l_word] / bi_gram_count[phrase]  # P(aW | W)
            le += p_aw_w * math.log2(p_aw_w)
        le = -le
        bi_gram_le[phrase] = le
    # 计算右熵
    for phrase in bi_gram_count:
        re = 0
        for r_word in bi_gram_rcount[phrase]:
            p_wa_w = bi_gram_rcount[phrase][r_word] / bi_gram_count[phrase]  # P(Wa | W)
            re += p_wa_w * math.log2(p_wa_w)
        re = -re
        bi_gram_re[phrase] = re
    # 计算PMI
    PMI = {}
    for phrase in bi_gram_count:
        p_first = word_count[phrase[0]] / all_word_count
        p_second = word_count[phrase[1]] / all_word_count
        p_bi_gram = bi_gram_count[phrase] / all_bi_gram_count
        PMI[phrase] = math.log2(p_bi_gram / (p_first * p_second))
    # 计算综合得分
    phrase_score = []
    for phrase in PMI:
        le = bi_gram_le[phrase]
        re = bi_gram_re[phrase]
        score = PMI[phrase] + le + re
        phrase_score.append((phrase, score))
    # 排序
    phrase_score = sorted(phrase_score, key=lambda x: x[1], reverse=True)
    return phrase_score

In [22]:
with open('../data(temp)/review_cleaned.txt', 'r',encoding='utf-8') as f:
        str_text = f.read()
with open('../data(temp)/review_seg_post.txt', 'r',encoding='utf-8') as f:
    seg_pos_text = f.readlines()

# 加载停用词
stop_word = []
with open('../data/stopword.txt', 'r',encoding='utf-8') as f:
    for line in f.readlines():
        stop_word.append(line.strip())

# 加载words_idf值
word_idf = {}
for idx, line in enumerate(open('../data(temp)/review_idf.txt', 'r',encoding='utf-8')):
    line = line.strip()
    line = line.split(' ')
    word_idf[line[0]] = float(line[1])

review_list, all_word = text2review(seg_pos_text)

phrase_score=find_word_phrase(all_word, review_list)
for i,item in enumerate(phrase_score):
        print('{}:{}'.format(''.join(item[0]), item[1]))
        if i>=39:
            break
print(phrase_score)
print(len(phrase_score))

细软塌:17.613342303095372
抖音:17.32570193918611
正装:16.963329808466835
越洗越:15.956687726653543
马龙代言:15.5570215576948
刘诗诗代言:15.54311687150312
都是:15.219602906716823
油性发质:15.134025837133327
不知道:15.062665787348788
掉发:14.581173405652873
油脂分泌:14.448134562966258
搞活动:14.434828652027722
让人:14.253156141721302
梳到底:14.138714645459146
能够有效:14.103285904124062
心情愉悦:14.0010990581482
一定要:13.81058904627797
颅顶:13.79850620838112
出油:13.73891998241529
比实体店:13.724457185000016
更加健康:13.703815468254895
买来试试:13.693275019940673
洗完头:13.622191652155418
惊艳到:13.577638534454454
一大瓶:13.564977841331647
发货速度:13.51434216709101
五星好评:13.498832308959447
收到货:13.389689443605675
能够深入:13.377891457806015
高颅:13.369009074198818
支持代言人:13.336484770967086
能有效:13.319910237608703
终于找到:13.292723486274234
贴着头皮:13.28357883185527
来评价:13.259722880586551
差评:13.200630441702245
再来:13.184126164395003
想象中:13.156597326261686
起头皮屑:13.145638719316882
不用洗头:13.11632395726181
[(('细软', '塌'), 17.613342303095372), (('抖', '音'), 17.32570193918611), (('正', '装'), 1

候选新词保存为csv

In [23]:
import csv  
# 将数据写入CSV文件  
with open('../results/PossibleNewword.csv', 'w', newline='', encoding='utf-8') as csvfile:  
    writer = csv.writer(csvfile)  
    writer.writerow(['合成词', '综合得分'])  # 写入表头  
    for (word1, word2), score in phrase_score:  
        composite_word = word1 + word2  # 合成词  
        writer.writerow([composite_word, f'{score:.4f}'])  # 写入合成词和格式化的得分  